<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Устанавливаем нужные зависимости

In [14]:
!pip install onnx transformers onnxruntime folium==0.2.1 optimum[onnxruntime]

     |████████████████████████████████| 69 kB 3.4 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=f9bf0db2d02a65f44c7bb11fdfbaee9793ebd7691e30fbe4310324beda448bb2
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


Фиксируем версии библиотек

In [15]:
!pip freeze > req.txt

Импорты

In [16]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer

Запускаем rubert как есть

In [17]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [18]:
%%time
predict('Правительство выделит 16 миллиардов рублей на поддержку клещей')
#вернулся нейтральный класс

CPU times: user 128 ms, sys: 46 µs, total: 128 ms
Wall time: 142 ms


0

Переводим в ONNX
1. Есть библиотека transforms, где все из коробки

https://huggingface.co/docs/transformers/serialization - делаем все по лучшим гайдам

In [19]:
class DistilBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        return OrderedDict(
            [
                ("input_ids", {0: "batch", 1: "sequence"}),
                ("attention_mask", {0: "batch", 1: "sequence"}),
                ("token_type_ids", {0: "batch", 1: "sequence"}),
            ]
        )

In [20]:
config = AutoConfig.from_pretrained("blanchefort/rubert-base-cased-sentiment")
onnx_config_for_seq_clf = DistilBertOnnxConfig(config, task="sequence-classification")
print(onnx_config_for_seq_clf.outputs)

OrderedDict([('logits', {0: 'batch'})])


In [21]:
!mkdir -p output/onnx_transforms

In [22]:
onnx_inputs, onnx_outputs = export(
        tokenizer,
        model,
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert.onnx"),
        opset=11)

Пробуем запустить в ONNX и посмотреть время инференса

In [23]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert.onnx", sess_options, providers)

Смотрим на входы и выходы импортированной модели

In [24]:
input_names = model_ONNX.get_inputs()
for input_name in input_names:
  print(input_name.name)

input_ids
attention_mask
token_type_ids


In [25]:
for on in model_ONNX.get_outputs():
  print(on.name)

logits


In [26]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  result  = model_ONNX.run(None, dict(inputs))
  predicted = result.index(max(result))
  return predicted

In [27]:
%%time
predict_onnx('Правительство выделит 16 миллиардов рублей на поддержку клещей')

CPU times: user 56.3 ms, sys: 0 ns, total: 56.3 ms
Wall time: 58.7 ms


0

Квантизация 

Делаем тоже по лучшим гайдам https://github.com/huggingface/optimum

Есть три вида квантизации - статическая, динамическая и Quantization-Aware-Training(QAT)

Динамическая квантизация не требует ничего, поэтому она самая простая

In [28]:
!mkdir -p output/quantization

In [29]:
model_checkpoint = "blanchefort/rubert-base-cased-sentiment"
# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained(model_checkpoint, feature="sequence-classification")

# Quantize the model!
quantizer.export(
    onnx_model_path="output/quantization/rubert.onnx",
    onnx_quantized_model_output_path="output/quantization/rubert-dyn-quantized.onnx",
    quantization_config=qconfig,
)

PosixPath('output/quantization/rubert-dyn-quantized.onnx')

Пробуем запустить динамечески квантизированную ONNX модель и посмотреть на время инференса

In [30]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("/content/output/quantization/rubert-dyn-quantized.onnx", sess_options, providers)

In [31]:
%%time
predict_onnx('Правительство выделит 16 миллиардов рублей на поддержку клещей')

CPU times: user 31.3 ms, sys: 4.81 ms, total: 36.1 ms
Wall time: 37.9 ms


0